# Solutions to homework week 9 - Deploy Models with AWS Lambda (Serverless)  


In [ ]:
# Download the model files from here: 
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

## Question 1

To be able to use this model, we need to know the name of the input and output nodes. 

What's the name of the output:

In [ ]:
import onnxruntime as ort

onnx_model_path = 'hair_classifier_v1.onnx'

session = ort.InferenceSession(
    onnx_model_path, providers=["CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name



In [ ]:
output_name

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?
    transforms.Resize((200, 200)),
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]

In [ ]:
# Preparing the image
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)

    return img

In [ ]:
img_raw = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')
img = prepare_image(img_raw, target_size=(200, 200))

In [ ]:
def preprocess(X):
    # rescale range to [0:1] (from values in [0, 255])
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


In [ ]:
import numpy as np

# Convert to numpy array
X_raw = np.array(img)

X = preprocess(X_raw)

#-- Images are represented as 3D arrays:
# - Height × Width × Channels
# - Channels: RGB (Red, Green, Blue)
# - Each channel: 8 bits (0-255 values)
X[0, 0, 0]

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [ ]:
def preprocess_pytorch(X):
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) to (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)

In [ ]:
from keras_image_helper import create_preprocessor

In [ ]:
preprocessor = create_preprocessor(preprocess_pytorch, target_size=(200, 200))

In [ ]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
X = preprocessor.from_url(url)

result = session.run([output_name], {input_name: X})
float_predictions = result[0][0].tolist()